In [2]:
from google.colab import files
uploaded = files.upload()

Saving bot_detection_data.csv to bot_detection_data.csv


In [1]:
import pandas as pd

In [2]:
data= pd.read_csv("bot_detection_data.csv")

In [3]:
data.head()

,User ID,Username,Tweet,Retweet Count,Mention Count,Follower Count,Verified,Bot Label,Location,Created At,Hashtags
0,132131,flong,Station activity person against natural majori...,85,1,2353,False,1,Adkinston,2020-05-11 15:29:50,NaN
1,289683,hinesstephanie,Authority research natural life material staff...,55,5,9617,True,0,Sanderston,2022-11-26 05:18:10,both live
2,779715,roberttran,Manage whose quickly especially foot none to g...,6,2,4363,True,0,Harrisonfurt,2022-08-08 03:16:54,phone ahead
3,696168,pmason,Just cover eight opportunity strong policy which.,54,5,2242,True,1,Martinezberg,2021-08-14 22:27:05,ever quickly new I
4,704441,noah87,Animal sign six data good or.,26,3,8438,False,1,Camachoville,2020-04-13 21:24:21,foreign mention


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm


In [ ]:
import re

def preprocess_text(text):
    """
    Clean and preprocess a tweet or text string.

    Steps:
    - Lowercase
    - Remove URLs
    - Remove mentions (@username)
    - Remove hashtags (#hashtag)
    - Remove special characters and digits except basic punctuation
    - Strip extra whitespace

    Args:
        text (str): Raw input text

    Returns:
        str: Cleaned text
    """
    # Lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags (just remove the # symbol but keep the word)
    text = re.sub(r'#', '', text)

    # Remove special characters (keep letters, numbers, basic punctuation)
    text = re.sub(r'[^a-z0-9\s.,!?\'\"]+', ' ', text)

    # Remove digits only if you want: (optional)
    # text = re.sub(r'\d+', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
data["Cleaned_Tweet"] = data["Tweet"].apply(preprocess_text)
print(data["Cleaned_Tweet"].head())


0    station activity person against natural majori...
1    authority research natural life material staff...
2    manage whose quickly especially foot none to g...
3    just cover eight opportunity strong policy which.
4                        animal sign six data good or.
Name: Cleaned_Tweet, dtype: object


In [ ]:
# Fill NaNs in tweet column
data["Cleaned_Tweet"] = data["Cleaned_Tweet"].fillna("")

# Extract features and labels
texts = data["Cleaned_Tweet"].tolist()
labels = data["Bot Label"].tolist()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta = RobertaModel.from_pretrained("roberta-base").to(device)
roberta.eval()


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [ ]:
def get_cls_embedding(text):
    tokens = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=64)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = roberta(**tokens)
    cls_emb = outputs.last_hidden_state[:, 0, :]  # CLS token
    return cls_emb.cpu()

embeddings = []
for text in tqdm(texts):
    emb = get_cls_embedding(text)
    embeddings.append(emb)

embeddings_tensor = torch.cat(embeddings, dim=0)
labels_tensor = torch.tensor(labels)


  5%|▍         | 2275/50000 [00:23<06:35, 120.72it/s]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    embeddings_tensor, labels_tensor, test_size=0.2, random_state=42)


In [ ]:
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=128, output_dim=2):
        super(BiLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, seq_len=1, input_dim)
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

model = BiLSTMClassifier().to(device)


In [ ]:
from sklearn.metrics import accuracy_score

optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

num_epochs = 25
batch_size = 16

train_losses = []
train_accuracies = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    for i in range(0, len(X_train), batch_size):
        batch_x = X_train[i:i+batch_size].to(device)
        batch_y = y_train[i:i+batch_size].to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch_y.cpu().tolist())

    epoch_loss = total_loss / (len(X_train) / batch_size)
    epoch_acc = accuracy_score(all_labels, all_preds)
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.plot(range(1, num_epochs+1), train_accuracies, marker='o')
plt.title("Training Accuracy per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(1, num_epochs+1), train_losses, marker='o', color='red')
plt.title("Training Loss per Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()


In [ ]:
model.eval()
y_pred = []

with torch.no_grad():
    for i in range(0, len(X_test), batch_size):
        batch_x = X_test[i:i+batch_size].to(device)
        outputs = model(batch_x)
        preds = torch.argmax(outputs, dim=1)
        y_pred.extend(preds.cpu().tolist())

print(classification_report(y_test, y_pred, digits=4))
